In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

In [25]:
!pip install tensorflow tensorflow_hub


1840.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
  Obtaining dependency information for tensorflow_hub from https://files.pythonhosted.org/packages/6e/1a/fbae76f4057b9bcdf9468025d7a8ca952dec14bfafb9fc0b1e4244ce212f/tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [19]:
# Load all the JSONs from a folder as a list of dictionaries

import json
import os

def load_jsons_from_folder(folder_path):
    jsons = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            with open(os.path.join(folder_path, filename)) as f:
                jsons.append(json.load(f))
    return jsons

labels = load_jsons_from_folder('./TMA WSI Labels/')


In [20]:
converted_data = [[(d['x'], d['y']) for d in sublist] for sublist in labels]
converted_data

[[(112.26139005113899, 814.5426545792653),
  (231.71985575839528, 803.6165201712869),
  (524.1983947119925, 798.7881964117091),
  (407.9957514604355, 801.9845990440785),
  (341.9564145658263, 800.3389355742296),
  (285.7270924236414, 799.4703160121644),
  (652.8819894055326, 792.6046694133804),
  (596.9497135582432, 795.4073838319541),
  (774.0922269909394, 787.2824988078206),
  (714.5975898429093, 786.6993759414676),
  (832.6654650324441, 778.3108627733717),
  (891.4752847246682, 778.1347892450393),
  (102.79473144030106, 752.588664613981),
  (333.790202342918, 742.879392971246),
  (281.8533644496305, 740.5718916115649),
  (463.9301246738185, 738.6201797622499),
  (525.0581529128752, 733.0226963706724),
  (713.9258602341256, 723.8452169800165),
  (650.2382978723405, 728.0791286727456),
  (596.1080470891791, 734.5020964360587),
  (778.2677902621723, 720.2086974615064),
  (337.9227903826617, 689.613839033751),
  (276.95633652822147, 687.0567092651758),
  (95.15991833957128, 693.35839854

In [21]:
# Get the paths of all the images in the folder as a list

import glob
image_paths = glob.glob('./TMA WSI PNGs/*.png')
image_paths.sort()
image_paths


['./TMA WSI PNGs/158867.png',
 './TMA WSI PNGs/158868.png',
 './TMA WSI PNGs/158869.png',
 './TMA WSI PNGs/158870.png',
 './TMA WSI PNGs/158871.png',
 './TMA WSI PNGs/ABC_096_1_009_1.png',
 './TMA WSI PNGs/ABC_097_1_009_1.png',
 './TMA WSI PNGs/ABC_098_1_009_1.png',
 './TMA WSI PNGs/ABC_099_1_009_1.png',
 './TMA WSI PNGs/ABC_100_1_009_1.png',
 './TMA WSI PNGs/ABC_101_1_009_1.png',
 './TMA WSI PNGs/ABC_102_1_009_1.png',
 './TMA WSI PNGs/ABC_103_1_009_1.png',
 './TMA WSI PNGs/ABC_104_1_009_1.png',
 './TMA WSI PNGs/ABC_105_1_010_1.png',
 './TMA WSI PNGs/ABC_106_1_010_1.png',
 './TMA WSI PNGs/ABC_107_1_010_1.png',
 './TMA WSI PNGs/ABC_108_1_010_1.png',
 './TMA WSI PNGs/ABC_109_1_010_1.png']

In [22]:
# Load and Preprocess the Dataset
# For the purpose of this example, we'll create a small sample dataset
# In practice, you should load your actual dataset here

# Sample dataset: list of dictionaries containing image file paths and core coordinates
sample_dataset = [
    {'image_path': image_paths[i], 'cores': converted_data[i]} for i in range(len(image_paths))
]

# Visualize the sample data
for data in sample_dataset:
    print(f"Image Path: {data['image_path']}")
    print(f"Cores Coordinates: {data['cores']}")

Image Path: ./TMA WSI PNGs/158867.png
Cores Coordinates: [(112.26139005113899, 814.5426545792653), (231.71985575839528, 803.6165201712869), (524.1983947119925, 798.7881964117091), (407.9957514604355, 801.9845990440785), (341.9564145658263, 800.3389355742296), (285.7270924236414, 799.4703160121644), (652.8819894055326, 792.6046694133804), (596.9497135582432, 795.4073838319541), (774.0922269909394, 787.2824988078206), (714.5975898429093, 786.6993759414676), (832.6654650324441, 778.3108627733717), (891.4752847246682, 778.1347892450393), (102.79473144030106, 752.588664613981), (333.790202342918, 742.879392971246), (281.8533644496305, 740.5718916115649), (463.9301246738185, 738.6201797622499), (525.0581529128752, 733.0226963706724), (713.9258602341256, 723.8452169800165), (650.2382978723405, 728.0791286727456), (596.1080470891791, 734.5020964360587), (778.2677902621723, 720.2086974615064), (337.9227903826617, 689.613839033751), (276.95633652822147, 687.0567092651758), (95.15991833957128, 69

In [23]:
# Prepare the Dataset for Training

# Creating a custom dataset class
class TissueMicroarrayDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = plt.imread(item['image_path'], format='jpg')  # This is a placeholder; replace with actual image loading
        image = self.pad_to_square(image)  # This is a placeholder; replace with actual image loading
        cores = item['cores']

        if self.transform:
            image = self.transform(image)

        return image, cores
    
    # Pads all the images to the same size (1024 x 1024). Ensure that the padding only occurs on the right of the image for width and bottom side of the image for height.
   
    def pad_to_square(self, img):
        # Get the height and width of the image
        h, w = img.shape[:2]

        # Pad only if the height and width are not equal
        if h != w:
            # Calculate the difference between the height and width
            diff = abs(h - w)

            # Pad the bottom of the image
            img = np.pad(img, ((0, diff), (0, 0), (0, 0)), 'constant')

        return img


# Splitting the dataset into training and validation sets
train_data = sample_dataset[:int(len(sample_dataset)*0.8)]  # 80% for training
valid_data = sample_dataset[int(len(sample_dataset)*0.8):]  # 20% for validation

# Creating data loaders
train_loader = TissueMicroarrayDataset(train_data)
valid_loader = TissueMicroarrayDataset(valid_data)

# Printing out one batch of data
for images, cores in train_loader:
    print(images)
    print(cores)

[[[243 243 243]
  [243 243 243]
  [243 243 243]
  ...
  [243 245 244]
  [243 245 244]
  [243 245 244]]

 [[243 243 243]
  [243 243 243]
  [243 243 243]
  ...
  [243 243 241]
  [243 243 241]
  [243 243 241]]

 [[243 243 243]
  [243 243 243]
  [243 243 243]
  ...
  [243 244 239]
  [243 244 239]
  [243 244 239]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
[(112.26139005113899, 814.5426545792653), (231.71985575839528, 803.6165201712869), (524.1983947119925, 798.7881964117091), (407.9957514604355, 801.9845990440785), (341.9564145658263, 800.3389355742296), (285.7270924236414, 799.4703160121644), (652.8819894055326, 792.6046694133804), (596.9497135582432, 795.4073838319541), (774.0922269909394, 787.2824988078206), (714.59

In [46]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import UpSampling2D

# Define the input shape
input_shape = (512, 512, 3)
inputs = Input(shape=input_shape)

# Load the base model
base_model = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/1", trainable=True)

# Use the base model on the inputs
cls_outputs, box_outputs = base_model(inputs)

# Concatenate the outputs
resized_outputs = [UpSampling2D(size=(64 // tensor.shape[1], 64 // tensor.shape[2]))(tensor) for tensor in box_outputs]
concatenated_outputs = Concatenate(axis=-1)(resized_outputs)

# Add additional layers
x = Dense(128, activation='relu')(concatenated_outputs)
x = Dropout(0.2)(x)

# Add the final output layer
outputs = Dense(2, activation='linear')(x)  # Output layer for X, Y coordinates

# Create the Model
model = Model(inputs=inputs, outputs=outputs)


In [47]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [48]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 keras_layer_8 (KerasLayer)  ([(None, 64, 64, 224),       1509201   ['input_8[0][0]']             
                              (None, 32, 32, 224),        6                                       
                              (None, 16, 16, 224),                                                
                              (None, 8, 8, 224),                                                  
                              (None, 4, 4, 224)],                                                 
                              [(None, 64, 64, 224),                                           